In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

import cv2
import logging
import time

import sys
sys.path.insert(0, '..')

from web_interaction.main import open_game, start_game, wait_loading
from game.game import ProductOwnerGame
from environment.backlog_env import BacklogEnv
from environment.environment import ProductOwnerEnv
from environment.reward_sytem.base_reward_system import BaseRewardSystem
from environment.userstory_env import UserstoryEnv

from pipeline.study_agent import load_dqn_agent

from web_interaction import GameImageParser, GameCoordinator, WebController

In [2]:
logger = logging.getLogger('WebInteraction')
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("%(asctime)s %(message)s"))
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

image_parser = GameImageParser('../web_interaction/templates')
game_coordinator = GameCoordinator(image_parser)
game = ProductOwnerGame()

userstory_env = UserstoryEnv(4, 0, 0)
backlog_env = BacklogEnv(12, 0, 0, 0, 0, 0)
reward_system = BaseRewardSystem(config={})
env = ProductOwnerEnv(userstory_env, backlog_env, with_info=True, reward_system=reward_system)
env.game = game

web_controller = WebController(game_coordinator, logger)

In [3]:
agent = load_dqn_agent('../models/credit_start_model.pt')

c:\Users\kruto\OneDrive\Рабочий стол\forks\product_owner_rl\web_interaction\..\pipeline\study_agent.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent: DQN = torch.l

In [4]:
driver = open_game()

iframe = driver.find_element(by=By.ID, value='game_drop')

# open full screen
fullscreen_button = driver.find_element(by=By.CLASS_NAME, value="fullscreen_btn")
fullscreen_button.click()

height = iframe.rect["height"]  # 540 or 1028
width = iframe.rect["width"]  # 960 or 1920

wait_loading(iframe)

start_game(driver, iframe)

iframe.screenshot('game_state.png')

image = cv2.imread('game_state.png')

In [5]:
game_coordinator.skip_tutorial(game)
game_coordinator.insert_user_stories_from_image(game, image)

game.userstories.stories_list

[UserStoryCard(S, loyalty=0.055, customers=1.0]

In [6]:
game_coordinator.update_header_info(game, image)

print('Sprint', game.context.current_sprint)
print('Money', game.context.get_money())
print('Loyalty', game.context.get_loyalty())
print('Customers', game.context.customers)
print('Credit', game.context.credit)

Sprint 4
Money 33000.0
Loyalty 4.0
Customers 25.0
Credit 300000


In [7]:
def apply_web_action(action: int, driver, iframe: WebElement, env: ProductOwnerEnv):
    if action == 0:  # start sprint
        web_controller.start_sprint(driver, iframe, env)
        return

    if action == 1:  # decompose
        web_controller.apply_decompose_action(driver, iframe, env)
        return

    if action == 2:  # release
        web_controller.release_tasks(driver, iframe, env)
        return

    if action == 5:  # buy statistical research
        web_controller.buy_statistical_research(driver, iframe, env)
        return

    if action >= env.meta_action_dim:

        action -= env.meta_action_dim

    if action < env.userstory_env.max_action_num:
        web_controller.apply_user_story_action(action, driver, iframe, env)
        return

    action -= env.userstory_env.max_action_num

    if action < env.backlog_env.backlog_max_action_num:
        web_controller.apply_backlog_card_action(action, driver, iframe, env)
        return

    raise Exception(f"Unknown action: {action}")

In [8]:
while not game.context.done:
    state = env.recalculate_state()
    # print(state)

    info = env.get_info()
    # print(info)

    action = agent.get_action(state, info)
    logger.info(f'Action id: {action}')

    time.sleep(1)
    apply_web_action(action, driver, iframe, env)

    if env.game.context.current_sprint >= 35:
        logger.warn("Reached credit end!")
        break

2024-11-04 13:55:24,216 Action id: 7
2024-11-04 13:55:25,219 Start user story action: 0
2024-11-04 13:55:25,221 User story: UserStoryCard(S, loyalty=0.055, customers=1.0
2024-11-04 13:55:25,225 Found at position: (1556, 384)
2024-11-04 13:55:27,318 Reward: True
2024-11-04 13:55:27,320 Action id: 1
2024-11-04 13:55:28,321 Start decomposition
2024-11-04 13:55:31,274 Action id: 16
2024-11-04 13:55:32,275 Start moving backlog card
2024-11-04 13:55:33,614 Selected card Card(CardInfo(15, (120, 79, 240), 1293219769552, S, UserCardType.S))
2024-11-04 13:55:33,616 Found at position (1516, 471)
2024-11-04 13:55:33,928 Clicked on card
2024-11-04 13:55:33,935 Action id: 17
2024-11-04 13:55:34,937 Start moving backlog card
2024-11-04 13:55:36,308 Selected card Card(CardInfo(1, (120, 79, 240), 1293219769552, S, UserCardType.S))
2024-11-04 13:55:36,312 Found at position (1516, 471)
2024-11-04 13:55:36,667 Clicked on card
2024-11-04 13:55:36,676 Action id: 0
2024-11-04 13:55:37,682 Start new sprint
20

In [ ]:
raise Exception('Break before empty sprinst')

In [9]:
while not game.context.done:
    time.sleep(1)
    apply_web_action(0, driver, iframe, env)

In [10]:
iframe.rect

{'height': 1028, 'width': 1920, 'x': 0, 'y': 0}

In [11]:
raise Exception('Break before web driver exit')

Exception: Break before web driver exit

In [ ]:
game_coordinator.backlog_cards

[BacklogCardImageInfo((54, 79, 234), 17, (1517, 384)),
 BacklogCardImageInfo((54, 79, 234), 9, (1598, 384)),
 BacklogCardImageInfo((54, 79, 234), 7, (1517, 471)),
 BacklogCardImageInfo((54, 79, 234), 5, (1598, 471)),
 BacklogCardImageInfo((84, 36, 195), 15, (1517, 558)),
 BacklogCardImageInfo((84, 36, 195), 11, (1598, 558)),
 BacklogCardImageInfo((84, 36, 195), 8, (1517, 645)),
 BacklogCardImageInfo((84, 36, 195), 4, (1598, 645))]

In [ ]:
game.userstories.release

[UserStoryCard(S, loyalty=0.065, customers=1.0,
 UserStoryCard(S, loyalty=0.06, customers=1.0]

In [ ]:
for us in game.userstories.release:
    print(us.info.color)

(54, 79, 234)
(120, 79, 240)


In [ ]:
game_coordinator.user_stories

[UserStoryImageInfo((120, 79, 240), 0.05, 2.0, (1556, 384)),
 UserStoryImageInfo((255, 211, 143), 0.06, 2.0, (1556, 471))]

In [ ]:
image_parser.templates = image_parser._load_templates()

In [ ]:
image = cv2.imread('game_state.png')

game_coordinator.update_header_info(game, image)

In [ ]:
info

{'actions': [0, 3, 4, 5, 6, 7, 8, 9, 10]}

In [ ]:
state

array([3.40e+01, 3.79e+00, 1.54e-02, 2.82e+00, 3.00e-01, 2.00e+00,
       1.00e+00, 0.00e+00, 0.00e+00, 1.30e+01, 1.00e+00, 0.00e+00,
       0.00e+00, 1.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e

In [ ]:
env.game.context.current_sprint

11

In [ ]:
driver.quit()